{/* cspell:ignore Lipinska mcscf chmax Dmax vmax ecore ncas Excp disp antisymmetry Coov irrep IIZZ ZZII workstreams openfermion webkitallowfullscreen allowfullscreen frameborder referrerpolicy Kassal Tranter Ardle Seeley */}

# Hamiltonians for Quantum Chemistry

Let's begin with a brief overview of the role Hamiltonians play in VQE.

## The Hamiltonian in VQE Overview

Dr. Victoria Lipinska walks us through Hamiltonians and how to map them for use in quantum computing.

<IBMVideo id="132414924" title="Victoria Lipinska reminds us what a Hamiltonian is and how it is used in VQE. In particular, she describes the need for mapping a fermionic Hamiltonian that governs indistinguishable fermions to a qubit Hamiltonian."/>

### References

The following articles are referenced in the above video.
* [Quantum Algorithms for Fermionic Simulations, Ortiz, et al.](https://arxiv.org/pdf/cond-mat/0012334.pdf)
* [Simulating Chemistry using Quantum Computers, Kassal et al.](https://arxiv.org/pdf/1007.2648.pdf)
* [A Comparison of the Bravyi–Kitaev and Jordan–Wigner Transformations for the Quantum Simulation of Quantum Chemistry, Tranter, et al.](https://pubs.acs.org/doi/full/10.1021/acs.jctc.8b00450)
* [Quantum Chemistry in the Age of Quantum Computing, Cao, et al.](https://arxiv.org/pdf/1812.09976.pdf)
* [Quantum computational chemistry, McArdle, et al.](https://arxiv.org/pdf/1808.10402.pdf)
* [The Bravyi-Kitaev transformation for quantum computation of electronic structure, Seeley, et al., McArdle, et al.](https://arxiv.org/pdf/1812.09976.pdf)

## Preparing Hamiltonians for Quantum Chemistry

A good first step in applying quantum computing to a chemistry problem is defining a Hamiltonian for the system of interest. Here, we will restrict the discussion to quantum chemistry Hamiltonians, as those Hamiltonians require some mapping specific to systems of identical fermions.

As someone working in quantum chemistry, you probably already have your favorite software for modeling molecules, which can generate a Hamiltonian that describes your system of interest. Here, we will use code built solely on PySCF, numpy, and Qiskit. But the process of Hamiltonian preparation transfers to prepackaged solutions as well. The only difference between this approach and other software will be minor syntax differences; some of these are addressed in the "Third-party software" subsection to facilitate integration of existing workflows.

Generating a quantum chemistry Hamiltonian for use on IBM Quantum&reg; QPUs involves the following steps:

1. Define your molecule (geometry, spin, active space, and so on)
2. Generate the fermionic Hamiltonian (creation and annihilation operators)
3. Map from the fermionic Hamiltonian to a bosonic operator (in this context, using Pauli operators)
4. If using third-party software: Handle any syntax mismatches between the generating software and Qiskit

The fermionic Hamiltonian is written in terms of fermionic operators, and in particular, takes into account that electrons are indistinguishable fermions. That means they obey completely different statistics from distinguishable, bosonic qubits. Hence the mapping process.

Those of you already familiar with these processes can likely skip this section.

Goal:

The end goal is to obtain a Hamiltonian of the form:

In [1]:
H = [(1, "XX"), (1, "YY"), (1, "ZZ")]
print(H)

[(1, 'XX'), (1, 'YY'), (1, 'ZZ')]


Or

In [2]:
from qiskit.quantum_info import SparsePauliOp

H = SparsePauliOp(["XX", "YY", "ZZ"], coeffs=[1.0 + 0.0j, 1.0 + 0.0j, 1.0 + 0.0j])
print(H)

SparsePauliOp(['XX', 'YY', 'ZZ'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j])


We'll start by importing some packages:

In [3]:
import numpy as np
from pyscf import ao2mo, gto, mcscf, scf

1. Define your molecule

Here we will specify attributes of the molecule of interest. In this example, we've chosen diatomic hydrogen (because the resulting Hamiltonians are short enough to display).

The Python-based Simulations of Chemistry Framework ([PySCF](https://pyscf.org/)) has a wide collection of electronic structure modules that can be used to, among other things, generate molecular Hamiltonians suitable for quantum computation. The [PySCF Quickstart](https://pyscf.org/quickstart.html) guide is an excellent resource for a full description of all the variables and functionality. We will give only the most cursory overview, since this will already be familiar to many of you. To understand these better, visit [PySCF.](https://pyscf.org/)
Briefly:

__distance__ can be used for diatomic molecules, or simply specify Cartesian coordinates for each atom. Distances are in units of Angstrom.

__gto__ generates gaussian-type orbitals.

__basis__ refers to the functions used to model molecular orbitals. Here 'sto-6g' is a common minimal basis, named for fitting Slater-Type Orbitals using 6 primitive Gaussian orbitals.

__spin__ an integer value indicating the number of unpaired electrons (equal to $2S$). Note that some software uses multiplicity instead ($2S+1$).

__charge__ the charge of the molecule.

__symmetry__ - the point symmetry group of the molecule, either specified with a string or automatically detected by setting "symmetry = True". Here "Dooh" is the appropriate symmetry group for diatomic molecules with two of the same atom species.

In [4]:
distance = 0.735
a = distance / 2
mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[
        ["H", (0, 0, -a)],
        ["H", (0, 0, a)],
    ],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry="Dooh",
)

Keep in mind that one can describe total energy (which includes nuclear repulsion energy as well as electronic), total electronic orbital energy, or the energy of some subset of electronic orbitals (with the complementary subset frozen). In the specific case of $\text{H}_2$, note the different energies below, and note that the total energy less the nuclear repulsion energy does in fact yield the electronic energy:

In [5]:
mf = scf.RHF(mol)
mf.scf()

print(
    mf.energy_nuc(),
    mf.energy_elec()[0],
    mf.energy_tot(),
    mf.energy_tot() - mol.energy_nuc(),
)

0.7199689944489797 -1.8455976628764188 -1.125628668427439 -1.8455976628764188


In [6]:
active_space = range(mol.nelectron // 2 - 1, mol.nelectron // 2 + 1)

2. Generate fermionic Hamiltonian

__scf__ refers to a wide range of self-consistent field methods.

__rhf__ as in __mf__ = scf.RHF(mol) in mf is a solver that uses the Restricted Hartree Fock calculation. The kernel of this (E, below) is the total energy, including nuclear repulsion and molecular orbitals.

__mcscf__ is a multi-configuration self-consistent fields package.

__ao2mo__ is a transformation from atomic orbitals to molecular orbitals.

We also use the following variables:

__ncas__: number of orbitals in the complete active space

__nelecas__: number of electrons in the complete active space

In [7]:
E1 = mf.kernel()
mx = mcscf.CASCI(mf, ncas=2, nelecas=(1, 1))
mo = mx.sort_mo(active_space, base=0)
E2 = mx.kernel(mo)[:2]

We want a Hamiltonian, and this is often separated into energy of an electronic core (ecore, not involved in minimization), single-electron operators (h1e), and two-electron energies (h2e). These are explicitly extracted below in the last two lines.

In [8]:
h1e, ecore = mx.get_h1eff()
h2e = ao2mo.restore(1, mx.get_h2eff(), mx.ncas)

These Hamiltonians are currently fermionic (creation and annihilation) operators, applicable to systems of (indistinguishable) fermions, and correspondingly subject to antisymmetry under exchange. This results in different statics than would apply to a distinguishable or bosonic system. To run calculations on IBM Quantum QPUs, we require a bosonic operator describing the energy. The result of such a mapping is conventionally written in terms of Pauli operators, since they are both Hermitian and unitary. There are several mappings one can use. One of the simplest is the Jordan Wigner transformation.

3. Mapping the Hamiltonian

It should be noted that there are many tools available for mapping a chemical Hamiltonian to one suitable for running on a quantum computer. Here, we implement the Jordan Wigner mapping directly using only PySCF, numpy, and Qiskit. We comment below on syntax considerations for other solutions.

The Cholesky function helps us obtain a low-rank decomposition of the two-electron terms in the Hamiltonian.

In [9]:
def cholesky(V, eps):
    # see https://arxiv.org/pdf/1711.02242.pdf section B2
    # see https://arxiv.org/abs/1808.02625
    # see https://arxiv.org/abs/2104.08957
    no = V.shape[0]
    chmax, ng = 20 * no, 0
    W = V.reshape(no**2, no**2)
    L = np.zeros((no**2, chmax))
    Dmax = np.diagonal(W).copy()
    nu_max = np.argmax(Dmax)
    vmax = Dmax[nu_max]
    while vmax > eps:
        L[:, ng] = W[:, nu_max]
        if ng > 0:
            L[:, ng] -= np.dot(L[:, 0:ng], (L.T)[0:ng, nu_max])
        L[:, ng] /= np.sqrt(vmax)
        Dmax[: no**2] -= L[: no**2, ng] ** 2
        ng += 1
        nu_max = np.argmax(Dmax)
        vmax = Dmax[nu_max]
    L = L[:, :ng].reshape((no, no, ng))
    print(
        "accuracy of Cholesky decomposition ",
        np.abs(np.einsum("prg,qsg->prqs", L, L) - V).max(),
    )
    return L, ng

The functions `identity` and `creators_destructors` replace creation and annihilation operators in the fermionic Hamiltonian with Pauli operators; `creators_destructors` uses the Jordan-Wigner mapping.

In [10]:
def identity(n):
    return SparsePauliOp.from_list([("I" * n, 1)])


def creators_destructors(n, mapping="jordan_wigner"):
    c_list = []
    if mapping == "jordan_wigner":
        for p in range(n):
            if p == 0:
                ell, r = "I" * (n - 1), ""
            elif p == n - 1:
                ell, r = "", "Z" * (n - 1)
            else:
                ell, r = "I" * (n - p - 1), "Z" * p
            cp = SparsePauliOp.from_list([(ell + "X" + r, 0.5), (ell + "Y" + r, 0.5j)])
            c_list.append(cp)
    else:
        raise ValueError("Unsupported mapping.")
    d_list = [cp.adjoint() for cp in c_list]
    return c_list, d_list

Finally, `build_hamiltonian` uses the `cholesky`, `identity`, and `creators_destructors` functions to create the final Hamiltonian suitable for running on a quantum computer.

In [11]:
def build_hamiltonian(ecore: float, h1e: np.ndarray, h2e: np.ndarray) -> SparsePauliOp:
    ncas, _ = h1e.shape

    C, D = creators_destructors(2 * ncas, mapping="jordan_wigner")
    Exc = []
    for p in range(ncas):
        Excp = [C[p] @ D[p] + C[ncas + p] @ D[ncas + p]]
        for r in range(p + 1, ncas):
            Excp.append(
                C[p] @ D[r]
                + C[ncas + p] @ D[ncas + r]
                + C[r] @ D[p]
                + C[ncas + r] @ D[ncas + p]
            )
        Exc.append(Excp)

    # low-rank decomposition of the Hamiltonian
    Lop, ng = cholesky(h2e, 1e-6)
    t1e = h1e - 0.5 * np.einsum("pxxr->pr", h2e)

    H = ecore * identity(2 * ncas)
    # one-body term
    for p in range(ncas):
        for r in range(p, ncas):
            H += t1e[p, r] * Exc[p][r - p]
    # two-body term
    for g in range(ng):
        Lg = 0 * identity(2 * ncas)
        for p in range(ncas):
            for r in range(p, ncas):
                Lg += Lop[p, r, g] * Exc[p][r - p]
        H += 0.5 * Lg @ Lg

    return H.chop().simplify()

Finally, we use `build_hamiltonian` to construct our qubit Hamiltonian from Pauli operators by using the Jordan-Wigner transformation. This also gives us the accuracy of the Cholesky decomposition we used.

In [12]:
H = build_hamiltonian(ecore, h1e, h2e)
print(H)

accuracy of Cholesky decomposition  2.220446049250313e-16
SparsePauliOp(['IIII', 'IIIZ', 'IZII', 'IIZI', 'ZIII', 'IZIZ', 'IIZZ', 'ZIIZ', 'IZZI', 'ZZII', 'ZIZI', 'YYYY', 'XXYY', 'YYXX', 'XXXX'],
              coeffs=[-0.09820182+0.j, -0.1740751 +0.j, -0.1740751 +0.j,  0.2242933 +0.j,
  0.2242933 +0.j,  0.16891402+0.j,  0.1210099 +0.j,  0.16631441+0.j,
  0.16631441+0.j,  0.1210099 +0.j,  0.17504456+0.j,  0.04530451+0.j,
  0.04530451+0.j,  0.04530451+0.j,  0.04530451+0.j])


This [example molecule notebook](https://ibm.box.com/shared/static/59c2kjqefvjy5wy5lct099thr4y0ry8c.zip) shows the setup and Hamiltonians for several molecules of varying complexity; with a little modification, this should enable you to examine most small molecules.

Let's just briefly note two important points to consider when constructing the fermionic operators for a molecule. As the molecule type changes, the symmetry will change. Similarly, the number of orbitals with various symmetries, like the cylindrically symmetric "A1", will change. These changes are evident even with the simple extension to LiH, as seen here:

In [13]:
distance = 1.56
mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[["Li", (0, 0, 0)], ["H", (0, 0, distance)]],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry="Coov",
)
mf = scf.RHF(mol)
E1 = mf.kernel()

# %% ----------------------------------------------------------------------------------------------

mx = mcscf.CASCI(mf, ncas=5, nelecas=(1, 1))
cas_space_symmetry = {"A1": 3, "E1x": 1, "E1y": 1}
mo = mcscf.sort_mo_by_irrep(mx, mf.mo_coeff, cas_space_symmetry)
E2 = mx.kernel(mo)[:2]
h1e, ecore = mx.get_h1eff()
h2e = ao2mo.restore(1, mx.get_h2eff(), mx.ncas)

It is also worth noting that one can quickly lose intuition for the final resulting Hamiltonian. The Hamiltonian for LiH (using the Jordan-Wigner mapper) already consists of 276 terms.

In [14]:
len(build_hamiltonian(ecore, h1e, h2e))

accuracy of Cholesky decomposition  1.1102230246251565e-16


276

When in doubt regarding symmetries, one can also generate some symmetry information for the molecule by setting `symmetry = True` and `verbose = 4`:

In [15]:
distance = 1.56
mol = gto.Mole()
mol.build(
    verbose=4,
    atom=[["Li", (0, 0, 0)], ["H", (0, 0, distance)]],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry=True,
)

System: uname_result(system='Linux', node='IBM-R912JTRT', release='5.10.102.1-microsoft-standard-WSL2', version='#1 SMP Wed Mar 2 00:30:59 UTC 2022', machine='x86_64')  Threads 16
Python 3.11.12 (main, May 16 2025, 02:33:32) [GCC 11.4.0]
numpy 2.3.1  scipy 1.16.0  h5py 3.14.0
Date: Mon Jun 30 12:56:55 2025
PySCF version 2.9.0
PySCF path  /home/porter284/.pyenv/versions/3.11.12/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 4
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry True subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 Li     0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.000000000000   1.560000000000 AA    0.000000000000   0.000

Among other useful information, this returns both `point group symmetry = Coov` and also the number of orbitals in each irreducible representation.

```
point group symmetry = Coov
num. orbitals of irrep A1 = 4
num. orbitals of irrep E1x = 1
num. orbitals of irrep E1y = 1
number of shells = 4
```

This does not necessarily tell you how many orbitals you want included in your active space, but it helps you see what orbitals are present and their symmetries.

Specifying symmetry and orbitals is often helpful, but you can also specify the number of orbitals you want to include. Consider the case of ethene, below. Using `verbose = 4`, we can print the symmetries of the various orbitals:

In [16]:
# Replace these variables with correct distances:
a = 1
b = 1
c = 1

# Build
mol = gto.Mole()
mol.build(
    verbose=4,
    atom=[
        ["C", (0, 0, a)],
        ["C", (0, 0, -a)],
        ["H", (0, c, b)],
        ["H", (0, -c, b)],
        ["H", (0, c, -b)],
        ["H", (0, -c, -b)],
    ],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry=True,
)

System: uname_result(system='Linux', node='IBM-R912JTRT', release='5.10.102.1-microsoft-standard-WSL2', version='#1 SMP Wed Mar 2 00:30:59 UTC 2022', machine='x86_64')  Threads 16
Python 3.11.12 (main, May 16 2025, 02:33:32) [GCC 11.4.0]
numpy 2.3.1  scipy 1.16.0  h5py 3.14.0
Date: Mon Jun 30 12:57:07 2025
PySCF version 2.9.0
PySCF path  /home/porter284/.pyenv/versions/3.11.12/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 16
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry True subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 C      0.000000000000   0.000000000000   1.000000000000 AA    0.000000000000   0.000000000000   1.889726124565 Bohr   0.0
[INPUT]  2 C      0.000000000000   0.000000000000  -1.000000000000 AA    0.000000000000   0.00

We obtain:

num. orbitals of irrep Ag = 4

num. orbitals of irrep B2g = 2

num. orbitals of irrep B3g = 1

num. orbitals of irrep B1u = 4

num. orbitals of irrep B2u = 1

num. orbitals of irrep B3u = 2

But rather than specifying all the orbitals by symmetry, we can simply write:

In [17]:
active_space = range(mol.nelectron // 2 - 2, mol.nelectron // 2 + 2)

In this approach, we take several orbitals near the filling level (valence and unoccupied). Here, 5 orbitals have been selected for inclusion in the active space (the 6th though 10th).

In [18]:
print(
    mol.nelectron // 2 - 2,
    mol.nelectron // 2 + 2,
)

6 10


4. Third-Party Software

There are several software packages developed for quantum chemistry, some offering multiple mappers and tools for restricting active spaces. The steps described above are general and apply to third-party software as well. But this other software may return Hamiltonians in a format that is not accepted by Qiskit. For example, some software returns Hamiltonians of the form:

`H = -0.042 [] + -0.045 [X0 X1 Y2 Y3] + ... + 0.178 [Z0] + ... + 0.176 [Z2 Z3] + -0.243 [Z3]`

Note in particular that the gates are numbered, and identity operators are not shown. This is in contrast to Hamiltonians used in Qiskit, which would write the term `[Z2 Z3]` as `ZZII` (qubits 0 and 1 being acting on by the identity operator, qubits 2 and 3 being acted on by the Z operator, ordered with qubit 0 furthest to the right).

To accommodate any existing workstreams you have, the code block below converts from one syntax to the other. The function `convert_openfermion_to_qiskit` takes as its arguments a Hamiltonian generated in OpenFermion or Tangelo (and already mapped onto Pauli operators using any available mapper), and the number of qubits needed for the molecule.

In [19]:
from openfermion import QubitOperator
from qiskit.quantum_info import SparsePauliOp


def convert_openfermion_to_qiskit(
    openfermion_operator: QubitOperator, num_qubits: int
) -> SparsePauliOp:
    terms = openfermion_operator.terms

    labels = []
    coefficients = []

    for term, constant in terms.items():
        # Default set to identity
        operator = list("I" * num_qubits)

        # Iterate through PauliSum and replace I with Pauli
        for index, pauli in term:
            operator[index] = pauli
        label = "".join(operator)
        labels.append(label)
        coefficients.append(constant)

    return SparsePauliOp(labels, coefficients)

Further, [this Python notebook](https://ibm.box.com/s/fh3xele1e7k0nrgd1imivvq52hy3wz9c) contains complete sample code for migrating Hamiltonians from other software workflows into Qiskit, including the conversion above.

You should now have an arsenal of tools for obtaining the Hamiltonian you need to perform quantum chemistry calculations on IBM&reg; quantum computers.